In [2]:
# Load models and preprocessor

import joblib
import numpy as np
import pandas as pd

# Load preprocessor
preprocessor = joblib.load("../models/preprocessor.joblib")

# Load trained models
dt_model = joblib.load("../models/decision_tree_model.joblib")
nn_model = joblib.load("../models/neural_network_model.joblib")


In [21]:
# User selects model

models_options = ["Decision Tree", "Neural Network"]

print("Select model for Churn Prediction:")
for i, m in enumerate(models_options, start=1):
    print(f"{i} - {m}")

while True:
    try:
        model_choice = int(input(
            "Select model for Churn Prediction:\n"
            "1 - Decision Tree\n"
            "2 - Neural Network\n"
            "Enter a number (1-2): "
        ))
        if model_choice == 1:
            selected_model_name = "Decision Tree"
            selected_model = dt_model
            break
        elif model_choice == 2:
            selected_model_name = "Neural Network"
            selected_model = nn_model
            break
        else:
            print("Invalid choice. Please try again.")
    except ValueError:
        print("Please enter a valid number.")
        
        
# Example: numeric_features = ['tenure','MonthlyCharges','TotalCharges','estimated_ltv']
# categorical_features = [all others except customerID]

# Collect user input

user_input = {}

# Numeric inputs
user_input['tenure'] = float(input("Enter tenure (months): "))
user_input['MonthlyCharges'] = float(input("Enter MonthlyCharges: "))
user_input['TotalCharges'] = float(input("Enter TotalCharges: "))

# Derived numeric feature
user_input['estimated_ltv'] = user_input['MonthlyCharges'] * user_input['tenure']

# Categorical features with predefined options
options = {
    'gender': ['Male', 'Female'],
    'SeniorCitizen': ['0', '1'],
    'Partner': ['Yes', 'No'],
    'Dependents': ['Yes', 'No'],
    'PhoneService': ['Yes', 'No'],
    'MultipleLines': ['Yes', 'No', 'No phone service'],
    'InternetService': ['DSL', 'Fiber optic', 'No'],
    'OnlineSecurity': ['Yes', 'No', 'No internet service'],
    'OnlineBackup': ['Yes', 'No', 'No internet service'],
    'DeviceProtection': ['Yes', 'No', 'No internet service'],
    'TechSupport': ['Yes', 'No', 'No internet service'],
    'StreamingTV': ['Yes', 'No', 'No internet service'],
    'StreamingMovies': ['Yes', 'No', 'No internet service'],
    'Contract': ['Month-to-month', 'One year', 'Two year'],
    'PaperlessBilling': ['Yes', 'No'],
    'PaymentMethod': ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
}

for col, opts in options.items():
    # Build prompt string
    prompt = f"Select {col} -\n"
    for i, opt in enumerate(opts, start=1):
        prompt += f"{i} - {opt}\n"
    prompt += f"Enter a number (1-{len(opts)}): "

    # Input validation loop
    while True:
        try:
            choice = int(input(prompt))
            if 1 <= choice <= len(opts):
                user_input[col] = opts[choice - 1]
                break
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Please enter a valid number.")

# Derived binary flags
user_input['has_internet'] = int(user_input['InternetService'] != 'No')
user_input['has_phone'] = int(user_input['PhoneService'] == 'Yes')

# Derived tenure group
tenure = user_input['tenure']
if tenure <= 12:
    user_input['tenure_group'] = '0-12'
elif tenure <= 24:
    user_input['tenure_group'] = '12-24'
elif tenure <= 48:
    user_input['tenure_group'] = '24-48'
elif tenure <= 60:
    user_input['tenure_group'] = '48-60'
else:
    user_input['tenure_group'] = '60-72'
    
#Convert input to dataframe
user_df = pd.DataFrame([user_input])

#Preprocess user input
X_user = preprocessor.transform(user_df)

# Predict churn
pred = selected_model.predict(X_user)[0]
pred_proba = selected_model.predict_proba(X_user)[0][1]

if pred == 1:
    print(f"\nPredicted Churn: YES ({pred_proba:.2f} probability) using {selected_model_name}")
else:
    print(f"\nPredicted Churn: NO ({1-pred_proba:.2f} probability) using {selected_model_name}")



Select model for Churn Prediction:
1 - Decision Tree
2 - Neural Network
Invalid choice. Please try again.

Predicted Churn: NO (0.82 probability) using Neural Network


### Result: User chooses which model, enters customer details, and sees churn prediction + probability.